In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import pandas as pd

In [2]:
# create new dataframe
raw_diet_df = pd.read_excel('diet.xls')
print(type(raw_diet_df))
print(raw_diet_df.shape)
raw_diet_df.columns = raw_diet_df.columns.str.replace(' ', '_')
raw_diet_df = raw_diet_df.replace(' ', '_', regex=True)

# print head
raw_diet_df.head(3)

<class 'pandas.core.frame.DataFrame'>
(67, 14)


,Foods,Price/_Serving,Serving_Size,Calories,Cholesterol_mg,Total_Fat_g,Sodium_mg,Carbohydrates_g,Dietary_Fiber_g,Protein_g,Vit_A_IU,Vit_C_IU,Calcium_mg,Iron_mg
0,Frozen_Broccoli,0.16,10_Oz_Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2_Cup_Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery,_Raw",0.04,1_Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2


In [3]:
# print tail, showing that the food & contraints information are mixed up
raw_diet_df.tail(5)

,Foods,Price/_Serving,Serving_Size,Calories,Cholesterol_mg,Total_Fat_g,Sodium_mg,Carbohydrates_g,Dietary_Fiber_g,Protein_g,Vit_A_IU,Vit_C_IU,Calcium_mg,Iron_mg
62,"Crm_Mshrm_Soup,W/Mlk",0.65,1_C_(8_Fl_Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn_Soup,W/Watr",0.67,1_C_(8_Fl_Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum_daily_intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum_daily_intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [4]:
# store constraints information only
constraints_df = raw_diet_df[65:].copy()
constraints_df.set_index('Serving_Size', drop=True, inplace=True)
constraints_df.rename(index={'Minimum_daily_intake':'Minimum'}, inplace=True)
constraints_df.rename(index={'Maximum_daily_intake':'Maximum'}, inplace=True)
constraints_df.drop(columns=['Foods', 'Price/_Serving'], inplace=True)

# convert df into dictionary
constraint_of = constraints_df.to_dict()
# constraint_of

In [5]:
# remove constraints information
diet_df = raw_diet_df[:64].copy()
diet_df.tail(3)

,Foods,Price/_Serving,Serving_Size,Calories,Cholesterol_mg,Total_Fat_g,Sodium_mg,Carbohydrates_g,Dietary_Fiber_g,Protein_g,Vit_A_IU,Vit_C_IU,Calcium_mg,Iron_mg
61,"New_E_Clamchwd,W/Mlk",0.99,1_C_(8_Fl_Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm_Mshrm_Soup,W/Mlk",0.65,1_C_(8_Fl_Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn_Soup,W/Watr",0.67,1_C_(8_Fl_Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [6]:
# store food types
diet_df.set_index('Foods', drop=True, inplace=True)
food_types = diet_df.index.values.tolist()

# store cost information
cost_of = diet_df['Price/_Serving'].to_dict()

# store nutrients information
diet_df.drop(columns=['Price/_Serving','Serving_Size'], inplace=True)
nutrient_dict = dict()
nutrients_list = diet_df.columns.values.tolist()
for nutrient in nutrients_list:
    nutrient_dict[nutrient] = diet_df[nutrient].to_dict()

# print(list( nutrient_dict.items() )[:1])

In [7]:
# initialize Linear Program with LpProblem
problem_cheapest = LpProblem("CheapestDiet", LpMinimize)

print(type(problem_cheapest))
print(problem_cheapest)

<class 'pulp.pulp.LpProblem'>
CheapestDiet:
MINIMIZE
None
VARIABLES



In [8]:
# add Variable with LpVariable.dicts
X = LpVariable.dicts("Units_serving_of", food_types, 0)

In [9]:
# add ObjectiveFunction with lpSum
problem_cheapest += lpSum([cost_of[i] * X[i] for i in food_types]), "TotalCost"

print(problem_cheapest)

CheapestDiet:
MINIMIZE
0.23*Units_serving_of_2%_Lowfat_Milk + 0.16*Units_serving_of_3.3%_Fat,Whole_Milk + 0.24*Units_serving_of_Apple,Raw,W_Skin + 0.16*Units_serving_of_Apple_Pie + 0.16*Units_serving_of_Bagels + 0.15*Units_serving_of_Banana + 0.67*Units_serving_of_Beanbacn_Soup,W_Watr + 0.15*Units_serving_of_Bologna,Turkey + 0.05*Units_serving_of_Butter,Regular + 0.31*Units_serving_of_Cap'N_Crunch + 0.07*Units_serving_of_Carrots,Raw + 0.04*Units_serving_of_Celery,_Raw + 0.25*Units_serving_of_Cheddar_Cheese + 0.28*Units_serving_of_Cheerios + 0.39*Units_serving_of_Chicknoodl_Soup + 0.03*Units_serving_of_Chocolate_Chip_Cookies + 0.28*Units_serving_of_Corn_Flks,_Kellogg'S + 0.39*Units_serving_of_Couscous + 0.65*Units_serving_of_Crm_Mshrm_Soup,W_Mlk + 0.27*Units_serving_of_Frankfurter,_Beef + 0.16*Units_serving_of_Frozen_Broccoli + 0.18*Units_serving_of_Frozen_Corn + 0.32*Units_serving_of_Grapes + 0.33*Units_serving_of_Ham,Sliced,Extralean + 0.83*Units_serving_of_Hamburger_W_Toppings + 0.31

In [10]:
# add Constraints with lpSum AND inequality signs
for nutrient in nutrients_list:
    minimum = constraint_of[nutrient]['Minimum']
    maximum = constraint_of[nutrient]['Maximum']
    problem_cheapest += lpSum([nutrient_dict[nutrient][i] * X[i] for i in food_types]) >= minimum, "Min" + nutrient
    problem_cheapest += lpSum([nutrient_dict[nutrient][i] * X[i] for i in food_types]) <= maximum, "Max" + nutrient

# enable print to show lengthy detailed information
# print(problem_cheapest)

In [11]:
# solve Diet Optimization problem
problem_cheapest.solve()
vars_solution = dict()
for v in problem_cheapest.variables():
    if v.varValue > 0:
        vars_solution[v.name] = v.varValue

# print(vars_solution)
# use pprint to nicely separate by line
print("Minimum cost from optimized solution: {:.3f}\n".format(value(problem_cheapest.objective)))
print("""Constraints:
1. Each nutrient is NOT less than minimum requirement,
2. Each nutrient is NOT more than maximum requirement.
""")
print("Variables selection from solution:\n")
import pprint
pprint.pprint(vars_solution)

Minimum cost from optimized solution: 4.337

Constraints:
1. Each nutrient is NOT less than minimum requirement,
2. Each nutrient is NOT more than maximum requirement.

Variables selection from solution:

{'Units_serving_of_Celery,_Raw': 52.64371,
 'Units_serving_of_Frozen_Broccoli': 0.25960653,
 'Units_serving_of_Lettuce,Iceberg,Raw': 63.988506,
 'Units_serving_of_Oranges': 2.2929389,
 'Units_serving_of_Poached_Eggs': 0.14184397,
 'Units_serving_of_Popcorn,Air_Popped': 13.869322}
